# Categorical Variables
* Drop Categorical Variables
* Label Encoding
* One-Hot Encoding

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('../../../Data/melbourne-housing-snapshot/melb_data.csv')
y = data.Price

In [4]:
X = data.drop(['Price'], axis=1)

In [5]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=0)

In [8]:
col_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]
X_train_full.drop(col_with_missing, axis=1, inplace=True)
X_valid_full.drop(col_with_missing, axis=1, inplace=True)

In [9]:
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype=='object']

In [12]:
low_cardinality_cols

['Type', 'Method', 'Regionname']

In [10]:
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

In [13]:
numerical_cols

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [14]:
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [15]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [16]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical Variables: ")
print(object_cols)

Categorical Variables: 
['Type', 'Method', 'Regionname']


In [23]:
# MAE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Drop Categorical Variables

In [25]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE 1: ")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE 1: 
175703.48185157913


## Label Encoding

In [26]:
from sklearn.preprocessing import LabelEncoder

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE 2: ")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE 2: 
165936.40548390493


## One-Hot Encoding 

In [28]:
from sklearn.preprocessing import OneHotEncoder

# object 컬럼들만 모아서 onehotencoder로 만듬.
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# index값 붙여줌.
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# object_cols drop
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# onehotvector 붙이기
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE 3: ")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE 3: 
166089.4893009678
